In [1]:
# -*- coding: utf-8 -*-

# ライブラリインポート
!pip install jpholiday -q

import jpholiday
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
"""ソフトバンクデータの読み込み・前処理を行う関数"""
def process_softbank_data(file_path):
    # データの読み込み
    softbank = pd.read_csv(file_path, encoding="utf-8")

    # 年月日を適切な形式に変換
    softbank["FormattedDate"] = pd.to_datetime(
        softbank["Year"].astype(str) + "/" +
        softbank["Date"].str.extract(r"(\d+)月(\d+)日")[0].fillna('0') +
        "/" + softbank["Date"].str.extract(r"(\d+)月(\d+)日")[1].fillna('0'),
        format="%Y/%m/%d"
    )

    # 曜日を追加
    softbank["Weekday"] = softbank["FormattedDate"].dt.day_name()

    return softbank

In [3]:
def process_weather_data(file_path):
    """天気データの読み込み・前処理を行う関数"""
    # データの読み込み
    weather = pd.read_csv(file_path, encoding="shift_jis")

    # 列名をリネーム
    weather.columns = [
        "yyyy/mm/dd",
        "Total_Precipitation (mm)",
        "Average_Temperature (℃)",
        "Average_wind_speed(m/s)"
    ]

    # Date列をdatetime型に変換
    weather["yyyy/mm/dd"] = pd.to_datetime(weather["yyyy/mm/dd"])

    return weather

In [4]:
def merge_datasets(softbank_df, weather_df):
    """ソフトバンクデータと天気データをマージする関数"""
    # マージ処理: 結合キーをdatetime型に合わせる
    merged_df = pd.merge(softbank_df, weather_df, how="left", left_on="FormattedDate", right_on="yyyy/mm/dd")
    return merged_df

In [5]:
def save_to_csv(df, output_path):
    """データフレームをCSVとして出力する関数"""
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [6]:
# ファイルパスの設定
softbank_file = r"..\data\softbank_audience_full_data.csv"
weather_file = r"..\data\weather.csv"
output_file = r"..\data\final_data.csv"

# 前処理の実行
softbank_df = process_softbank_data(softbank_file)
weather_df = process_weather_data(weather_file)

# データのマージ
df = merge_datasets(softbank_df, weather_df)

In [7]:
df.head(3)

,Year,Date,Audience,Result,Score,Opponent,Pitcher,GameTime,Venue,FormattedDate,Weekday,yyyy/mm/dd,Total_Precipitation (mm),Average_Temperature (℃),Average_wind_speed(m/s)
0,2015,3月27日(金),38500,●,1 - 3,ロッテ,攝津,3:17,ヤフオクドーム,2015-03-27,Friday,2015-03-27,12.2,0.0,1.7
1,2015,3月28日(土),37397,○,4 - 2,ロッテ,スタンリッジ,2:37,ヤフオクドーム,2015-03-28,Saturday,2015-03-28,16.1,0.0,2.2
2,2015,3月29日(日),38118,●,4 - 5,ロッテ,中田,3:21,ヤフオクドーム,2015-03-29,Sunday,2015-03-29,16.1,0.0,2.5


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Year                      714 non-null    int64         
 1   Date                      714 non-null    object        
 2   Audience                  714 non-null    int64         
 3   Result                    714 non-null    object        
 4   Score                     714 non-null    object        
 5   Opponent                  714 non-null    object        
 6   Pitcher                   704 non-null    object        
 7   GameTime                  704 non-null    object        
 8   Venue                     714 non-null    object        
 9   FormattedDate             714 non-null    datetime64[ns]
 10  Weekday                   714 non-null    object        
 11  yyyy/mm/dd                714 non-null    datetime64[ns]
 12  Total_Precipitation (m

In [13]:
def preprocess_data(df):
    # "中止"データを排除
    df = df[df['Score'] != '中止'].copy()  # copy()を追加して警告を回避

    # 該当のドームのみ
    df = df[df["Venue"].isin(['ヤフオクドーム', 'PayPayドーム', 'みずほPayPay'])].copy()

    # 日付の処理
    df['Date'] = pd.to_datetime(df['FormattedDate'])
    df['Weekday'] = pd.to_datetime(df['FormattedDate']).dt.day_name()

    # 結果の数値化
    df['Result'] = df['Result'].apply(lambda x: 0 if x == '●' else 1)

    # スコアの分割
    df[['Home_Score', 'Away_Score']] = df['Score'].str.split(' - ', expand=True)
    df['Home_Score'] = pd.to_numeric(df['Home_Score'])
    df['Away_Score'] = pd.to_numeric(df['Away_Score'])

    # 観客数の数値化
    df['Audience'] = pd.to_numeric(df['Audience'])

    # 降水量、気温、雲量の数値化
    df['Total_Precipitation (mm)'] = pd.to_numeric(df['Total_Precipitation (mm)'])
    df['Average_Temperature (℃)'] = pd.to_numeric(df['Average_Temperature (℃)'])
    df[ "Average_wind_speed(m/s)"] = pd.to_numeric(df[ "Average_wind_speed(m/s)"])

    # 対戦チームをエンコード
    n = df["Weekday"].nunique()
    df = pd.get_dummies(df, columns=['Weekday'], drop_first=False)
    # 最後のn列をint型に変換
    df.iloc[:, -n:] = df.iloc[:, -n:].astype(int)

    # 曜日をエンコード
    n = df["Opponent"].nunique()
    df = pd.get_dummies(df, columns=['Opponent'], drop_first=False)
    # 最後のn列をint型に変換
    df.iloc[:, -n:] = df.iloc[:, -n:].astype(int) 

    # ゲーム時間の分に変換
    df['GameTime'] = df['GameTime'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

    # 不要な列の削除
    df.drop(columns=['FormattedDate', 'Score','yyyy/mm/dd', "Venue"], inplace=True)

    return df

df_preprocessed = preprocess_data(df)

In [14]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 651 entries, 0 to 713
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Year                      651 non-null    int64         
 1   Date                      651 non-null    datetime64[ns]
 2   Audience                  651 non-null    int64         
 3   Result                    651 non-null    int64         
 4   Pitcher                   651 non-null    object        
 5   GameTime                  651 non-null    int64         
 6   Total_Precipitation (mm)  651 non-null    float64       
 7   Average_Temperature (℃)   651 non-null    float64       
 8   Average_wind_speed(m/s)   651 non-null    float64       
 9   Home_Score                651 non-null    int64         
 10  Away_Score                651 non-null    int64         
 11  Weekday_Friday            651 non-null    int32         
 12  Weekday_Monday            6

In [15]:

df_preprocessed.describe()

,Year,Date,Audience,Result,GameTime,Total_Precipitation (mm),Average_Temperature (℃),Average_wind_speed(m/s),Home_Score,Away_Score,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Opponent_DeNA,Opponent_オリックス,Opponent_ヤクルト,Opponent_ロッテ,Opponent_中日,Opponent_巨人,Opponent_広島,Opponent_日本ハム,Opponent_楽天,Opponent_西武,Opponent_阪神
count,651.000000,651,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000
mean,2019.476190,2019-12-28 16:37:36.221198336,30670.645161,0.634409,195.792627,23.903687,6.582181,2.814439,4.274962,3.397849,0.167435,0.038402,0.188940,0.185868,0.132104,0.136713,0.150538,0.018433,0.178187,0.023041,0.181260,0.018433,0.018433,0.023041,0.168971,0.178187,0.168971,0.023041
min,2015.000000,2015-03-27 00:00:00,0.000000,0.000000,132.000000,10.200000,0.000000,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.000000,2017-06-24 12:00:00,30369.500000,0.000000,177.000000,20.500000,0.000000,2.200000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019.000000,2019-09-18 00:00:00,35805.000000,1.000000,195.000000,24.000000,0.000000,2.600000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2022.000000,2022-07-02 00:00:00,38500.000000,1.000000,212.000000,28.100000,4.000000,3.200000,6.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2024.000000,2024-10-04 00:00:00,40178.000000,1.000000,301.000000,32.300000,231.500000,7.100000,22.000000,17.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,2.886142,NaN,12097.404856,0.481966,27.894334,4.853749,19.184263,0.869539,2.975743,2.768492,0.373651,0.192314,0.391762,0.389299,0.338865,0.343808,0.357873,0.134615,0.382965,0.150151,0.385529,0.134615,0.134615,0.150151,0.375014,0.382965,0.375014,0.150151


In [17]:
save_to_csv(df_preprocessed, r"..\data\df_preprocessed.csv")